In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import re
import joblib as jb

In [2]:
like = jb.load("like.pkl.z")
train = jb.load("train.pkl.z")
test = jb.load("test.pkl.z")

tag_like = jb.load("tag_like.pkl.z")
tag_bias_factor = jb.load("tag_bias_factor.pkl.z")

tag_pvals = jb.load("tag_pvals.pkl.z")
tag_bias_factor_weighted = jb.load("tag_bias_factor_weighted.pkl.z")

# features - avg

In [57]:
#avg bias feature

def gen_features(data, tag_dict, return_y=True):
    feature_col = []
    Y = []

    for ts, tags, y in data:

        feature_row = []

        for tag in tags:
            if tag not in tag_dict:
                feature_row.append(1)
                continue
            feature_row.append(tag_dict[tag])
            
        feature_col.append(np.mean(feature_row))
        Y.append(y)
        
    feature_col = np.array(feature_col)
    feature_col[np.isnan(feature_col)] = 1. # nan = 1.
    feature_col = feature_col.reshape(-1,1) #sklearn
    
    Y = np.array(Y)
        
    if return_y:
        return feature_col, Y
    return feature_col

# Models

In [58]:
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

## Tag Like

In [60]:
feature_col_tag_like_tr, Y_tr = gen_features(train, tag_like)
feature_col_tag_like_ts, Y_ts = gen_features(test, tag_like)

mdl = LGBMClassifier(random_state=0)
mdl.fit(feature_col_tag_like_tr, Y_tr)

raw_roc = roc_auc_score(Y_ts, feature_col_tag_like_ts)
model_roc = roc_auc_score(Y_ts, mdl.predict_proba(feature_col_tag_like_ts)[:,1])
print("ROC AUC - Feature = {} - Model = {}".format(raw_roc, model_roc))

/Users/mariofilho/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/mariofilho/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ROC AUC - Feature = 0.8828576104225871 - Model = 0.8856311290653981


## Tag Bias Factor

In [61]:
feature_col_tag_bias_factor_tr, Y_tr = gen_features(train, tag_bias_factor)
feature_col_tag_bias_factor_ts, Y_ts = gen_features(test, tag_bias_factor)

mdl = LGBMClassifier(random_state=0)
mdl.fit(feature_col_tag_bias_factor_tr, Y_tr)

raw_roc = roc_auc_score(Y_ts, feature_col_tag_bias_factor_ts)
model_roc = roc_auc_score(Y_ts, mdl.predict_proba(feature_col_tag_bias_factor_ts)[:,1])
print("ROC AUC - Feature = {} - Model = {}".format(raw_roc, model_roc))

/Users/mariofilho/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/mariofilho/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ROC AUC - Feature = 0.8605241884273186 - Model = 0.8725653372975385


## Tag P-values

In [7]:
feature_col_tag_pvals_tr, Y_tr = gen_features(train, tag_pvals)
feature_col_tag_pvals_ts, Y_ts = gen_features(test, tag_pvals)

mdl = LGBMClassifier(random_state=0)
mdl.fit(feature_col_tag_pvals_tr, Y_tr)

raw_roc = roc_auc_score(Y_ts, feature_col_tag_pvals_ts)
model_roc = roc_auc_score(Y_ts, mdl.predict_proba(feature_col_tag_pvals_ts)[:,1])
print("ROC AUC - Feature = {} - Model = {}".format(raw_roc, model_roc)) ## maior p-val, mais chance de ser de qualidade .... sample size? popularidade da tag?

/Users/mariofilho/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/mariofilho/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ROC AUC - Feature = 0.6775924316381104 - Model = 0.7288903811961154


## Tag Bias Factor Weighted

In [62]:
feature_col_tag_bias_factor_weighted_tr, Y_tr = gen_features(train, tag_bias_factor_weighted)
feature_col_tag_bias_factor_weighted_ts, Y_ts = gen_features(test, tag_bias_factor_weighted)

mdl = LGBMClassifier(random_state=0)
mdl.fit(feature_col_tag_bias_factor_weighted_tr, Y_tr)

raw_roc = roc_auc_score(Y_ts, feature_col_tag_bias_factor_weighted_ts)
model_roc = roc_auc_score(Y_ts, mdl.predict_proba(feature_col_tag_bias_factor_weighted_ts)[:,1])
print("ROC AUC - Feature = {} - Model = {}".format(raw_roc, model_roc))

/Users/mariofilho/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/mariofilho/miniconda3/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ROC AUC - Feature = 0.8441389961082729 - Model = 0.8625036086374923


## Tag Likes + Tag P-values

In [63]:
mx_tr = np.hstack([feature_col_tag_like_tr, feature_col_tag_pvals_tr])
mx_ts = np.hstack([feature_col_tag_like_ts, feature_col_tag_pvals_ts])

mdl = LGBMClassifier(random_state=0)
mdl.fit(mx_tr, Y_tr)

raw_roc = roc_auc_score(Y_ts, mx_ts.mean(axis=1))
model_roc = roc_auc_score(Y_ts, mdl.predict_proba(mx_ts)[:,1])
print("ROC AUC - Feature = {} - Model = {}".format(raw_roc, model_roc))

ROC AUC - Feature = 0.8639511544581207 - Model = 0.8895922280392354


# Matriz de Tag Likes

In [10]:
from scipy.sparse import csr_matrix

In [65]:
def like_to_matrix(raw_data, tag_dict):
    data = []
    rows = []
    cols = []
    Y = []
    
    col_names = sorted(tag_dict.keys())
    col_ix = dict(zip(col_names, np.arange(len(col_names))))

    for i, (ts, tags, y) in enumerate(raw_data):
        for t in tags:
            row = i
            col = col_ix.get(t, -1)
            if col < 0:
                continue
            value = tag_dict[t]
            #print(value)

            data.append(value)
            rows.append(i)
            cols.append(col)
        Y.append(y)
        #break

    mx = csr_matrix((data, (rows, cols)), shape=(len(raw_data), len(col_names)))
    Y = np.array(Y)
    return mx, Y

In [66]:
mx_tr, Y_tr = like_to_matrix(train, tag_like)
mx_ts, Y_ts = like_to_matrix(test, tag_like)

mdl = LGBMClassifier(random_state=0)
mdl.fit(mx_tr, Y_tr)

raw_roc = roc_auc_score(Y_ts, mx_ts.mean(axis=1))
model_roc = roc_auc_score(Y_ts, mdl.predict_proba(mx_ts)[:,1])
print("ROC AUC - Feature = {} - Model = {}".format(raw_roc, model_roc))
# pvals pior, média com zeros, leva em conta numero de tags, pode ser outra feature

ROC AUC - Feature = 0.7873495980993808 - Model = 0.8681349453103238


In [56]:
mx_tr, Y_tr = like_to_matrix(train, tag_pvals)
mx_ts, Y_ts = like_to_matrix(test, tag_pvals)

mdl = LGBMClassifier(random_state=0)
mdl.fit(mx_tr, Y_tr)

raw_roc = roc_auc_score(Y_ts, mx_ts.mean(axis=1))
model_roc = roc_auc_score(Y_ts, mdl.predict_proba(mx_ts)[:,1])
print("ROC AUC - Feature = {} - Model = {}".format(raw_roc, model_roc))
# pvals pior, média com zeros, leva em conta numero de tags, pode ser outra feature

ROC AUC - Feature = 0.542390528697607 - Model = 0.7942591414316815
